In [ ]:
!pip3 install fasttext
!pip3 install transformers

In [ ]:
!git clone https://github.com/AI4Bharat/IndicLID.git

In [ ]:
%cd "/content/IndicLID/Inference"

In [ ]:
%mkdir models
%cd "/content/IndicLID/Inference/models"

In [ ]:
!wget https://github.com/AI4Bharat/IndicLID/releases/download/v1.0/indiclid-bert.zip
!wget https://github.com/AI4Bharat/IndicLID/releases/download/v1.0/indiclid-ftn.zip
!wget https://github.com/AI4Bharat/IndicLID/releases/download/v1.0/indiclid-ftr.zip

In [ ]:
!unzip indiclid-bert.zip
!unzip indiclid-ftn.zip
!unzip indiclid-ftr.zip

In [ ]:
# %cd "/content/IndicLID/"
%cd "/content/IndicLID/Inference"

In [ ]:
from ai4bharat.IndicLID import IndicLID

IndicLID_model = IndicLID(input_threshold = 0.5, roman_lid_threshold = 0.6)

In [ ]:
import json

# Function to extract all 'body' fields from comments and their replies
def extract_bodies(data):
    bodies = []
    for item in data:  # Assuming the top-level structure is a list
        if 'comments' in item:  # Check if the 'comments' key exists
            for comment in item['comments']:
                if 'body' in comment:  # Check if the 'body' key exists in the comment
                    clean_body = comment['body'].replace('\n', ' ')  # Replace newline characters with a space
                    bodies.append(clean_body)
                if 'reply' in comment:  # Check if there are replies
                    for reply in comment['reply']:
                        if 'body' in reply:  # Check if the 'body' key exists in the reply
                            clean_reply_body = reply['body'].replace('\n', ' ')  # Similarly, clean up the reply body
                            bodies.append(clean_reply_body)
    return bodies

# Function to read JSON data from a file and extract bodies
def read_and_extract(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:  # Open the file in read mode
        data = json.load(file)  # Load JSON data from the file
        return extract_bodies(data)  # Extract bodies using the function defined earlier

# Specify the path to your JSON file
file_path = '/content/MergedDelhi.json'  # Update this to the path of your JSON file
comment_bodies = read_and_extract(file_path)
print(comment_bodies)
test_samples1 = comment_bodies
seen = set()
unique_list = []
for item in test_samples1:
    if item not in seen:
        unique_list.append(item)
        seen.add(item)


In [111]:
test_samples = ['hi i am rebekah, i go to school in India, I am 22 years old', 'mera nam rebekah hai, apka kya hai', 'hi bro'
]

In [141]:
batch_size = 1
outputs = IndicLID_model.batch_predict(unique_list, batch_size)

In [ ]:
print(outputs)

Below code to get a list of only detected english comments

In [ ]:
english_list = []
count=0
for item in outputs:
    if item[1] == 'eng_Latn':
        #print(item[0])
        english_list.append(item)
        count = count + 1
print(count)

Removing stopwords and emojis

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

import re

def remove_emojis(text):
    # Unicode ranges to match typical emojis
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F700-\U0001F77F"  # alchemical symbols
        u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA00-\U0001FA6F"  # Chess Symbols
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U00002702-\U000027B0"  # Dingbats
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)  # Replacing emojis with empty string

# Initialize the English stopwords set
stop_words = set(stopwords.words('english'))

# Process the data using indices
cleaned_data = []
for item in english_list:
    if item[1] == 'eng_Latn':
        text = remove_emojis(item[0])
        tokens = word_tokenize(text)
        filtered_text = ' '.join(word for word in tokens if word.lower() not in stop_words)
        cleaned_data.append((filtered_text, item[1]))



Bar plot of frequency of predicted languages

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter


# Extracting the language and script codes
languages = [item[1] for item in outputs]

# Counting the frequency of each language
language_counts = Counter(languages)

# Creating labels and values for the plot
labels = list(language_counts.keys())
values = list(language_counts.values())

# Creating the bar chart
plt.figure(figsize=(10, 5))
plt.bar(labels, values, color='yellow')
plt.xlabel('Language')
plt.ylabel('Frequency')
plt.title('Frequency of Predicted Languages')
plt.xticks(rotation=45)
plt.show()


Checks for any duplicates in data

In [ ]:
from collections import Counter
temp=0

# Extracting only the text snippets
texts = [item[0] for item in outputs]

# Counting the frequency of each text snippet
text_counts = Counter(texts)

# Printing the text snippets that appear more than once
for text, count in text_counts.items():
    if count > 1:
        temp = temp + 1
print(temp)


Word Cloud of all the identified english text

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Combine all text snippets into one large string
text = " ".join([item[0] for item in cleaned_data])

# Generate the word cloud
wordcloud = WordCloud(width = 800, height = 400, background_color ='white').generate(text)

# Display the word cloud using matplotlib
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()


Table with count and percentages

In [ ]:
from collections import Counter


# Extracting the language codes
languages = [item[1] for item in outputs]

# Counting occurrences of each language
language_counts = Counter(languages)

# Calculating total number of entries for percentage calculation
total = sum(language_counts.values())

# Creating a sorted list of tuples from the language_counts dictionary
sorted_languages = sorted(language_counts.items(), key=lambda x: x[1], reverse=True)

# Calculating percentage and printing results
print("Language\tCount\tPercentage")
for language, count in sorted_languages:
    percentage = (count / total) * 100
    print(f"{language}\t{count}\t{percentage:.2f}%")

